# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
# **Análise Visual e Interativa Sobre o Discurso de Ódio nas Redes Sociais e Influência nas Eleições Brasileiras**
#### *Prof <sup>as</sup>. Viviane P. Moreira e Lisiane Selau*
<br>

---

***Trabalho de Conclusão de Curso – Turma 03***

*Aluno:*

* Antonio Fagner Santos de Oliveira 576637
---

_Este notebook trabalha análise exploratória dos dados das eleições gerais brasileiras de 2018 e 2022._

_Na análise exploratória devem-se usar técnicas estatísticas e visualização, seão identificadas tendências, padrões, relações e insights iniciais que podem orientar as próximas etapas do processo._

## 1 Carga dos Dados Pré-processados

In [1]:
import polars as pl
import os
import altair as alt
import numpy as np
import standards as sdt_f

In [2]:
root:str = "../Dados/Eleicoes/"
poll_18:str = "eleicao18_turno_01.parquet"
poll_22:str = "eleicao22_turno_01.parquet"
parties:str = "partidos_br.parquet"

In [3]:
#carga dos partidos políticos
df_partidos:pl.DataFrame = sdt_f.load_parquet(os.path.join(root, parties))

In [4]:
#Carregas dados das eleições de 2018 e 2022 já ajustados
df_poll_18 = sdt_f.load_parquet(os.path.join(root, poll_18))
df_poll_22 = sdt_f.load_parquet(os.path.join(root, poll_22))

## 2 Análise Exploratória

Entender e explorar os dados por meio de técnicas estatísticas e visualização. Tendências, padrões, relações e insights iniciais que podem orientar as próximas etapas do processo.

https://ebaconline.com.br/blog/analise-exploratoria-de-dados-o-que-e

---

### 2.1 Classificação Ideológica dos Partidos Políticos Brasileiros Conforme _Bolognesi[2018]_

---

In [5]:
df_colors = sdt_f.parties_colors(df_partidos) #define uma cor para cada partido político baseado na sua ideologia

In [6]:
'''
    Plota, na reta numérica, os partidos conforme suas ideologias políticas.
    Baseado no trabalho de Bolognesi(2018)

    df_partidos:pl.DataFrame -> Dataframe com os partidos políticos e sua classificação ideológica
    
'''
def class_ideologica_chart(df_partidos:pl.DataFrame, df_colors:pl.DataFrame)->alt.vegalite.v5.api.Chart:
    df_colors_tmp = df_colors.sort(by="MEDIA_IDEOL") 
    _domains = df_colors_tmp.get_column("SG_PARTIDO").to_list()
    _range = df_colors_tmp.get_column("cor").to_list()
    color_scale = alt.Scale(
        domain=_domains,
        range=_range
    )   
    
    #svg que renderiza uma pessoinha
    person = (
        "M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 "
        "-0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 "
        "0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 "
        "0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 "
        "0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 "
        "-0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 "
        "-0.6 -0.4 -0.6z"
    )
    
    points = alt.Chart(
        df_colors, 
        title="Classificação ideológica dos partidos políticos brasileiros"
    ).mark_point(
        filled=True,
        size=100,
        opacity=.6
    ).encode(
        x=alt.X("MEDIA_IDEOL:Q", 
                title="Média Ideológica",
                scale=alt.Scale(nice=True)
        ),
        shape=alt.ShapeValue(person),
        color=alt.Color('SG_PARTIDO:O',
            scale=color_scale, legend=None
        ),
        tooltip=[
            alt.Tooltip("SG_PARTIDO:O", title="Partido"),
            alt.Tooltip("MEDIA_IDEOL:Q", title="Média Ideológica", format=".2f",)
        ]        
    )
    text = points.mark_text(
        align="left",
        baseline="top",
        dy=-3,
        dx=30,
        fontSize=8,
        angle=270,
        fontWeight="bold",
        opacity=1
    ).encode(
        text=alt.Text("SG_PARTIDO:O"),
        tooltip=[
            alt.Tooltip("SG_PARTIDO:O", title="Partido"),
            alt.Tooltip("MEDIA_IDEOL:Q", title="Média Ideológica", format=".2f",)
        ] 
    )
    
    return (points + text) \
    .properties(
        width=1000,
        height=45
    ).configure_view(
        strokeWidth=0
    ).configure_title(
        fontSize=14
    )

In [7]:
class_ideologica_chart(df_partidos, df_colors)

alt.LayerChart(...)

_Através de um survey aplicado à comunidade de cientistas políticos em 2018, Bolognesi(2018) pediu que classificassem os partidos na dimensão esquerda-direita e também quanto ao seu principal objetivo: a persecução de votos, de posições de governo ou de políticas._

_Os resultados apontam para um movimento centrífugo do sistema partidário, com a maioria dos partidos caminhando para a direita, e para o predomínio de partidos que podem ser classificados como fisiológicos, priorizando a díade votos-cargos e desprezando a programaticidade._

### 2.2 Variação da votação por partidos e cargos entre 2018 e 2022

---

In [8]:
df_poll_18

ANO_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,DS_CARGO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_VALIDOS,SIGLA_2022,POSIC_IDEOLOGICO,SG_POSIC_IDEOLOGICO,SG_REGIAO,NM_REGIAO
i64,str,str,str,str,i64,str,i64,i64,str,i64,str,str,i64,str,str,str,str,str
2018,"""E""","""PE""","""PE""","""PERNAMBUCO""",23132,"""ALIANÇA""",125,7,"""Deputado Estadual""",77,"""SOLIDARIEDADE""","""Solidariedade""",148,"""SOLIDARIEDADE""","""Centro Direita""","""CD""","""NE""","""Nordeste"""
2018,"""E""","""PE""","""PE""","""PERNAMBUCO""",26077,"""TORITAMA""",112,7,"""Deputado Estadual""",77,"""SOLIDARIEDADE""","""Solidariedade""",32,"""SOLIDARIEDADE""","""Centro Direita""","""CD""","""NE""","""Nordeste"""
2018,"""E""","""PE""","""PE""","""PERNAMBUCO""",24490,"""ITAÍBA""",143,7,"""Deputado Estadual""",77,"""SOLIDARIEDADE""","""Solidariedade""",38,"""SOLIDARIEDADE""","""Centro Direita""","""CD""","""NE""","""Nordeste"""
2018,"""E""","""PE""","""PE""","""PERNAMBUCO""",24015,"""CUSTÓDIA""",65,7,"""Deputado Estadual""",77,"""SOLIDARIEDADE""","""Solidariedade""",15,"""SOLIDARIEDADE""","""Centro Direita""","""CD""","""NE""","""Nordeste"""
2018,"""E""","""PE""","""PE""","""PERNAMBUCO""",23817,"""CARUARU""",106,7,"""Deputado Estadual""",77,"""SOLIDARIEDADE""","""Solidariedade""",212,"""SOLIDARIEDADE""","""Centro Direita""","""CD""","""NE""","""Nordeste"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018,"""E""","""TO""","""TO""","""TOCANTINS""",95591,"""PORTO NACIONAL""",3,6,"""Deputado Federal""",11,"""PP""","""Partido Progressista""",237,"""PP""","""Direita""","""D""","""N""","""Norte"""
2018,"""E""","""TO""","""TO""","""TOCANTINS""",73261,"""PAU D ARCO""",31,6,"""Deputado Federal""",11,"""PP""","""Partido Progressista""",748,"""PP""","""Direita""","""D""","""N""","""Norte"""
2018,"""E""","""TO""","""TO""","""TOCANTINS""",96792,"""RIO SONO""",23,6,"""Deputado Federal""",11,"""PP""","""Partido Progressista""",12,"""PP""","""Direita""","""D""","""N""","""Norte"""


In [9]:
'''
   Apresenta um grouped bar chart para um cargo político comparando os 15 partidos que mais
   conseguiram votos em 2022 com 2018.
   df_2018:pl.DataFrame -> dados da eleição de 2018
   df_2022:pl.DataFrame -> dados da eleição de 2022
   df_colors:pl.DataFrame -> Lista dos partidos com sua classificação ideológica
   position:int -> código do cargo eleitoral (1 Presidente, 3 Governador, 5 Senador, 6 Deputado Federal,
                                               7 Deputado Estadual e 8 Deputado Distrital)
   chart_title:str -> título do chart

   Retorna um chart vega-altair   
'''

def grouped_bar_chart(df_2018:pl.DataFrame, df_2022:pl.DataFrame, df_colors:pl.DataFrame, 
                      position:list[int], chart_title:str, total_parties:int=5)->alt.vegalite.v5.api.Chart:
    # soma todos os votos por partidos de 2018
    df_tmp_18 = sdt_f.filter_by_position(df_2018, position)

    df_tmp_18 = (df_tmp_18
        .group_by(["ANO_ELEICAO", "SIGLA_2022","SG_POSIC_IDEOLOGICO"], maintain_order=True)
        .agg(pl.col("QT_VOTOS_VALIDOS").sum())
        .join(df_colors, on="SG_POSIC_IDEOLOGICO", how="inner")
    )

    # soma todos os votos por partidos de 2022
    df_tmp_22 = sdt_f.filter_by_position(df_2022, position)
    
    df_tmp_22 = (df_tmp_22
        .group_by(["ANO_ELEICAO", "SIGLA_2022","SG_POSIC_IDEOLOGICO"], maintain_order=True)
        .agg(pl.col("QT_VOTOS_VALIDOS").sum())
        .join(df_colors, on="SG_POSIC_IDEOLOGICO", how="inner")
    )

    # concatena as votações de 22 e 18 para os cargos em position
    df_tmp = pl.concat([df_tmp_22, df_tmp_18])

    # os registros dos K partidos mais votados em 2018
    top_k_22=df_tmp_22.top_k(total_parties, by="QT_VOTOS_VALIDOS").select(pl.col("SIGLA_2022","QT_VOTOS_VALIDOS"))    

    # faz join entre com os 15 partidos mais votados em 22 usando o campo QT_ORDER 
    # para ordenar o chart
    df_tmp = (df_tmp
        .join(top_k_22, on="SIGLA_2022", how="inner")
        .with_columns(
            QT_ORDER = pl.col("QT_VOTOS_VALIDOS_right")
        )
        .drop(pl.col("QT_VOTOS_VALIDOS_right"))              
    )  

    #plota o grouped bar chart
    return alt.Chart(df_tmp).mark_bar().encode(
        y = alt.Y('ANO_ELEICAO:O', 
            title="",
        ),
        x = alt.X('QT_VOTOS_VALIDOS:Q', title="Votação", axis=alt.Axis( labelAngle=-90 )),
        color = alt.Color('ANO_ELEICAO:O',
            scale = alt.Scale(
               domain=[2018,2022], 
               range=['#F58518','#4C78A8'],
            ),
            legend=None
        ),
        row = alt.Row('SIGLA_2022:N',
            title="",
            spacing = 5,
            header=alt.Header(labelAngle=0, labelAlign='left', title=f'{chart_title}', titleFontSize=12),
            sort=alt.SortField("QT_ORDER", order="descending"),                      
        ),
        opacity=alt.OpacityValue(0.75),
        tooltip=[
            alt.Tooltip('SIGLA_2022:N', title="Partido"),
            alt.Tooltip('ANO_ELEICAO:O', title="Ano Eleição"),
            alt.Tooltip('QT_VOTOS_VALIDOS:Q', format=",d",  title="Votos"),
        ]        
    ).properties(
        width=300,
        #height=150,
    ) 

grouped_bar_chart(df_poll_18, df_poll_22, df_colors, list([7,8]), 'Deputado Estadual/Distrital - 2018/2022')

alt.Chart(...)

In [10]:
df_colors

SG_PARTIDO,SG_POSIC_IDEOLOGICO,MEDIA_IDEOL,cor
str,str,f32,str
"""PSB""","""CE""",4.05,"""#C54B53"""
"""PV""","""C""",5.29,"""#FFD966"""
"""PPL""","""D""",7.27,"""#262DDA"""
"""PSD""","""CD""",7.09,"""#97A3FF"""
"""CIDADANIA""","""C""",4.92,"""#FFD966"""
…,…,…,…
"""PODE""","""D""",7.24,"""#262DDA"""
"""AVANTE""","""CD""",6.32,"""#97A3FF"""
"""DC""","""D""",8.11,"""#262DDA"""
